In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from torch import cuda, bfloat16
import transformers
import os
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=bfloat16
# )

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Need auth token for these
hf_auth = os.environ.get('hf_token')
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

In [3]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Given a n3.pcap file containing network traffic data, create a Python script using scapy library to process the PCAP. The script should extract the source IP, destination IP, source port, destination port, and the raw payload data for each packet in the PCAP. Store this data in a pandas DataFrame and display it."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1012)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Given a n3.pcap file containing network traffic data, create a Python script using scapy library to process the PCAP. The script should extract the source IP, destination IP, source port, destination port, and the raw payload data for each packet in the PCAP. Store this data in a pandas DataFrame and display it. [/INST]  Sure! Here is an example of how you can use the `scapy` library to process a PCAP file and extract the desired information in Python:
```
import scapy
import pandas as pd

# Load the PCAP file
pcap = scapy.load("n3.pcap")

# Extract the source IP, destination IP, source port, destination port, and payload data for each packet
packets = [pcap.packet for p in pcap.packets]
source_ips = [p.src for p in packets]
destination_ips = [p.dst for p in packets]
source_ports = [p.srcport for p in packets]
destination_ports = [p.dstport for p in packets]
payloads = [p.payload for p in packets]

# Create a pandas DataFrame with the extracted information
df = pd.DataFrame({

# 5G General Knowledge

In [ ]:
import json
import re

input_file_path = "/home/hb/LLM-research/evaluation/5G_SPEC/5G_multi_choice.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]
n_question = 0

for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["choices"]
    
    # Construct prompt with question and options
    prompt = f"Choose the correct answer: {question}\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
    match = re.search(r'(Answer:|Correct answer:)\s*((?:[A-D]\)\s*)?[\w\s()\-/]+)', generated_answer)
    if match:
        parsed_answer = match.group(2).strip()
        
        # Exclude the "Explanation" part from the parsed answer if present
        explanation_start = parsed_answer.find("\nExplanation")
        if explanation_start != -1:
            parsed_answer = parsed_answer[:explanation_start].strip()
        
        print(f"Parsed_answer: {parsed_answer}")
        generated_answers.append(parsed_answer)
    else:
        print("No answer detected.")
        generated_answers.append("N/A")
    
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print(f"GPT answer: {generated_answer}. Correct answer: {ground_answer}")
        print("Incorrect")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

In [ ]:
import json

input_file_path = "/home/hb/LLM-research/evaluation/5G_SPEC/5G_SPEC_QA.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

generated_answers_qa = []

n_question = 0
for q_dict in questions_data:
    n_question += 1
    question = q_dict["question"]
    # Generate answer using the language model
    prompt = f"[INST]{question}[/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Output: {generated_answer}")
    generated_answers_qa.append(generated_answer)
    print(f"---------------------{n_question}-----------------------------")

`Network config QA`: Pass

# 5G analysis knowledge

In [11]:
import json
import re

input_file_path = "/home/hb/LLM-research/evaluation/5G/5G_analysis_multi_choice_v2.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]
n_question = 0

for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["options"]
    
    # Construct prompt with question and options
    prompt = f"Choose the correct answer. {question}\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
    # Parse the generated answer
    answer_lines = generated_answer.splitlines()
    parsed_answer = ""
    for line in answer_lines:
            if line.startswith("Answer: ") or line.startswith("Correct answer: "):
                match = re.search(r'\b(\w)[\)-]', line)
                if match is not None:
                    parsed_answer = match.group(1)
                    print(f"Parsed_answer: {parsed_answer}")
                    break
                else:
                    parsed_answer = "Couldn't parse the answer"
                    print(f"Parsed_answer: {parsed_answer}")
    generated_answers.append(parsed_answer)
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print(f"GPT answer: {generated_answer}. Correct answer: {ground_answer}")
        print("Incorrect")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. What happens to packets larger than the Maximum SDU size in the network?
a) Always dropped
b) Forwarded with increased QoS
c) Implementation specific
d) Always forwarded with no changes

Answer: b) Forwarded with increased QoS

Explanation: When a packet is larger than the Maximum SDU size, it is broken down into smaller packets called PDUs (Packet Data Units). These PDUs are then forwarded through the network with increased QoS (Quality of Service) to ensure that
Parsed_answer: b
----------------1-----------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. The Maximum Transfer Unit (MTU) of the IP layer and the Maximum SDU Size:
a) Are directly related
b) Have no relationship
c) Are the same
d) Depend on the GGSN

Answer: a) Are directly related

Explanation: The Maximum Transfer Unit (MTU) of the IP layer and the Maximum SDU Size are directly related. The MTU of the IP layer determines the maximum size of the Segmentation Data Unit (SDU) that can be transmitted over an IP network.
Parsed_answer: a
----------------2-----------------


...

Generated Answer: Choose the correct answer. For which reason might the PGW reject an IP-CAN session modification procedure after an SGW failure?
a) If the request includes changing the PDN connection route
b) If the request includes the installation/modification of PCC rules
c) If the PGW has already initiated a new SGW restoration process
d) If the MME/SSGSN has already restored the PDN connection

Answer: a) If the request includes changing the PDN connection route.

Explanation: The PGW may reject an IP
Parsed_answer: a
----------------27-----------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. What does OMR stand for in the context of the provided text?
Optimized Media Routeing
Operational Management Review
Online Music Recorder
Optical Message Reading
Answer: B - Operational Management Review.

Explanation: In the context of the provided text, OMR stands for Operational Management Review, which is a process used to evaluate and improve the efficiency and effectiveness of an organization's operations.
Parsed_answer: Couldn't parse the answer
----------------28-----------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. What protocol is mentioned alongside SIP for OMR based signaling flows?
HTTP
SMTP
SDP
UDP

Answer: SDP (Session Description Protocol) is mentioned alongside SIP for OMR based signaling flows.

Explanation: OMR (On-Media Resource) is a mechanism that allows for the signaling of media resources on the client-side, without the need for a separate signaling protocol. SIP (Session Initiation Protocol) is used for initiating, modifying, and terminating sessions, but it does not provide the
Parsed_answer: Couldn't parse the answer
----------------29-----------------
Generated Answer: Choose the correct answer. What is the role of IBCF in the provided context?
Allocating MR context
Decoding audio files
Compressing video streams
Routing email traffic
IBCF (Intelligent Bridging and Control Function) is a network function that provides intelligent bridging and control of network traffic. It is used to manage and direct traffic in a network, based on va

## Multiple Choice
`5G configuration analysis v1`:
Accuracy: 66%
Correct answers: 13 + 7 = 20
Incorrect answers: 10

`5G configuration analysis v2`:
Accuracy: 56%
Correct answers: 17
Incorrect answers: 13